#### BASELINE SOLUTION

A simple baseline method could be a naive method, where the forecast for the next week would be simply equal to the usage of the previous week.

In [ ]:
# To install required modules for running the program
pip install -r requirements.txt

First, I'll load the data and preprocess it to get the hourly bus usages for each municipality, by aggregating the two measurements for an hour and taking the maximum value:

In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('municipality_bus_utilization.csv')

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Aggregate the data by hour
df_hourly = df.groupby(['municipality_id', pd.Grouper(key='timestamp', freq='H')]).max().reset_index()
df_hourly

,municipality_id,timestamp,usage,total_capacity
0,0,2017-06-04 07:00:00,204,2813
1,0,2017-06-04 08:00:00,332,2813
2,0,2017-06-04 09:00:00,485,2813
3,0,2017-06-04 10:00:00,583,2813
4,0,2017-06-04 11:00:00,614,2813
...,...,...,...,...
6735,9,2017-08-19 12:00:00,849,1332
6736,9,2017-08-19 13:00:00,941,1332
6737,9,2017-08-19 14:00:00,983,1332
6738,9,2017-08-19 15:00:00,976,1332


Next, I'll split the data into training and test sets, where the last two weeks (i.e., the last 336 hours) will be used as the test set:

In [2]:
# Create training and test sets
train_end = pd.to_datetime('2017-08-05')
test_start = pd.to_datetime('2017-08-06')
test_end = pd.to_datetime('2017-08-19')
train = df_hourly[df_hourly['timestamp'] < train_end].copy()
test = df_hourly[(df_hourly['timestamp'] >= test_start) & (df_hourly['timestamp'] <= test_end)].copy()

#### TRAIN DATA

In [3]:
train.head()

,municipality_id,timestamp,usage,total_capacity
0,0,2017-06-04 07:00:00,204,2813
1,0,2017-06-04 08:00:00,332,2813
2,0,2017-06-04 09:00:00,485,2813
3,0,2017-06-04 10:00:00,583,2813
4,0,2017-06-04 11:00:00,614,2813


#### TEST DATA

In [4]:
test.head()

,municipality_id,timestamp,usage,total_capacity
541,0,2017-08-06 07:00:00,280,2813
542,0,2017-08-06 08:00:00,391,2813
543,0,2017-08-06 09:00:00,459,2813
544,0,2017-08-06 10:00:00,569,2813
545,0,2017-08-06 11:00:00,653,2813


If there is a missing data then

In [5]:
# Fill missing data with the last valid observation
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

Now, I can make our forecast using the simple baseline method, which is to simply use the previous week's usage as the forecast for the next week:

In [6]:
# Simple baseline: take the hourly maximum of the last week of training data as the prediction for the test data
baseline_pred = train[train['timestamp'] >= pd.to_datetime('2017-07-29')].groupby('municipality_id')['usage'].max().reset_index()
baseline_pred = pd.merge(test[['municipality_id']], baseline_pred, on='municipality_id', how='left')
baseline_pred['usage'] = baseline_pred['usage'].fillna(0)
baseline_pred

,municipality_id,usage
0,0,688
1,0,688
2,0,688
3,0,688
4,0,688
...,...,...
1235,9,946
1236,9,946
1237,9,946
1238,9,946


Finally, I can calculate the error between the forecast and the true values for the test set:

In [7]:
# Calculate error for the last two weeks of test data
test_last_week = test[test['timestamp'] >= test_end - pd.Timedelta(days=7)]
test_error = np.sqrt(((test_last_week['usage'] - baseline_pred['usage']) ** 2).mean())

print("Simple baseline test error: {:.2f}".format(test_error))

Simple baseline test error: 1775.37


This will display the root mean squared error (RMSE), which is a gauge of how well our forecast performs, between the forecast and the test set's true values.